# 1. Crawler

## 1.0. Related example

This code shows `wget`-like tool written in python. Run it from console (`python wget.py`), make it work. Check the code, reuse, and modify for your needs.

In [1]:
# installing required libraries 
import sys

!{sys.executable} -m pip install bs4
!{sys.executable} -m pip install regex
!{sys.executable} -m pip install argparse
!{sys.executable} -m pip install requests
!{sys.executable} -m pip install urllib
!{sys.executable} -m pip install nltk

In [2]:
# downloading stopwords
import nltk
import ssl

try:
    _create_unverified_https_context = ssl._create_unverified_context
except AttributeError:
    pass
else:
    ssl._create_default_https_context = _create_unverified_https_context

nltk.download()

In [3]:
# create temp directory for storing downloaded html files
import os

os.mkdir('temp')

In [1]:
import argparse
import os
import re
import requests

def wget(url, filename):
    # allow redirects - in case file is relocated
    resp = requests.get(url, allow_redirects=True)
    # this can also be 2xx, but for simplicity now we stick to 200
    # you can also check for `resp.ok`
    if resp.status_code != 200:
        print(resp.status_code, resp.reason, 'for', url)
        return
    
    # just to be cool and print something
    print(*[f"{key}: {value}" for key, value in resp.headers.items()], sep='\n')
    print()
    
    # try to extract filename from url
    if filename is None:
        # start with http*, ends if ? or # appears (or none of)
        m = re.search("^http.*/([^/\?#]*)[\?#]?", url)
        filename = m.group(1)
        if not filename:
            raise NameError(f"Filename neither given, nor found for {url}")

    # what will you do in case 2 websites store file with the same name?
    if os.path.exists(filename):
        raise OSError(f"File {filename} already exists")
    
    with open(filename, 'wb') as f:
        f.write(resp.content)
        print(f"File saved as {filename}")


# if __name__ == "__main__":
#     parser = argparse.ArgumentParser(description='download file.')
#     parser.add_argument("-O", type=str, default=None, dest='filename', help="output file name. Default -- taken from resource")
#     parser.add_argument("url", type=str, default=None, help="Provide URL here")
#     args = parser.parse_args()
#     wget(args.url, args.filename)

### 1.0.1. How to parse a page?

If you build a crawler, you might follow one of the approaches:
1. search for URLs in the page, assuming this is just a text.
2. search for URLs in the places where URLs should appear: `<a href=..`, `<img src=...`, `<iframe src=...` and so on.

To follow the first approach you can rely on some good regular expression. [Like this](https://stackoverflow.com/a/3809435).

To follow the second approach just read one of these: [short answer](https://stackoverflow.com/questions/1080411/retrieve-links-from-web-page-using-python-and-beautifulsoup) or [exhaustive explanation](https://hackersandslackers.com/scraping-urls-with-beautifulsoup/).

## 1.1. [15] Download and persist #
Please complete a code for `load()`, `download()` and `persist()` methods of `Document` class. What they do:
- for a given URL `download()` method downloads binary data and stores in `self.content`. It returns `True` for success, else `False`.
- `persist()` method saves `self.content` somewhere in file system. We do it to avoid multiple downloads (for caching in other words).
- `load()` method loads data from hard drive. Returns `True` for success.

Tests checks that your code somehow works.

**NB Passing the test doesn't mean you correctly completed the task.** These are **criteria, which have to be fullfilled**:
1. URL is a unique identifier (as it is a subset of URI). Thus, documents with different URLs should be stored in different files. Typical errors: documents from the same domain are overwritten to the same file, URLs with similar endings are downloaded to the same file, etc.
2. The document can be not only a text file, but also a binary. Pay attention that if you download `mp3` file, it still can be played. Hint: don't hurry to convert everything to text.

In [2]:
import hashlib
import requests
from urllib.parse import quote

class Document:
    
    def __init__(self, url):
        self.url = url
        self.content = None
        
    def get(self):
        if not self.load():
            if not self.download():
                raise FileNotFoundError(self.url)
            else:
                self.persist()
    
    def download(self):
        resp = requests.get(self.url, allow_redirects=True)
        if resp.status_code != 200:
            return False
        else:
            self.content = resp.content
            return True
    
    def persist(self):
        f = open(f'temp/{quote(self.url).replace("/", "_")}',"wb")
        f.write(self.content)
        f.close()
        pass
            
    def load(self):
        try:
            f = open(f'temp/{quote(self.url).replace("/", "_")}',"rb")
            self.content = f.read()
            f.close()
        except:
            return False
        return True

In [3]:
quote("http://sprotasov.ru/data/iu.txt").replace("/", "_") 

'http%3A__sprotasov.ru_data_iu.txt'

### 1.1.1. Tests ###

In [4]:
doc = Document('http://sprotasov.ru/data/iu.txt')

doc.get()
assert doc.content, "Document download failed"
assert "Code snippets, demos and labs for the course" in str(doc.content), "Document content error"

doc.get()
assert doc.load(), "Load should return true for saved document"
assert "Code snippets, demos and labs for the course" in str(doc.content), "Document load from disk error"

## 1.3. [10] Parse HTML ##
`BeautifulSoap` library is a de facto standard to parse XML and HTML documents in python. Use it to complete `parse()` method that extracts document contents. You should initialize:
- `self.anchors` list of tuples `('text', 'url')` met in a document. Be aware, there exist relative links (e.g. `../content/pic.jpg`). Use `urllib.parse.urljoin()` to fix this issue.
- `self.images` list of images met in a document. Again, links can be relative to current page.
- `self.text` should keep plain text of the document without scripts, tags, comments and so on. You can refer to [this stackoverflow answer](https://stackoverflow.com/a/1983219) for details.

**NB All these 3 criteria must be fulfilled to get full point for the task.**

In [5]:
from bs4 import BeautifulSoup, Doctype
from bs4.element import Comment
import urllib.parse

class HtmlDocument(Document):

    def _visible_text(self, element):
        if isinstance(element, Comment):
            return False
        if element.parent.name in ['style', 'title', 'head', 'script', '[document]']:
            return False
        return True
    
    def _normalize_link(self, link):
        if link is not None and link[:4] == 'http':
            return link
        return urllib.parse.urljoin(self.url, link)
    
    def _get_anchors(self):
        soup = BeautifulSoup(self.content)
        self.anchors = []
        a_links = soup.find_all("a")
        for a in a_links:
            if a.has_attr('href'):
                text = a.getText()
                link = a.get('href')
                self.anchors.append((text, self._normalize_link(link)))
    
    def _get_images(self):
        soup = BeautifulSoup(self.content)
        self.images = []
        imgs = soup.find_all("img")
        for img in imgs:
            link = img.get('src')
            self.images.append(self._normalize_link(link))
            
    def _get_text(self):
        soup = BeautifulSoup(self.content)
        for item in soup.contents:
            if isinstance(item, Doctype):
                item.extract()
                
        text_list = filter(self._visible_text, soup.findAll(text=True))
        self.text = " ".join([word.strip() for word in text_list]).replace("\n", "").strip()

        # self.text = soup.find("body").getText().replace("\n", "").strip()
    
    def parse(self):
        self.get()
        self._get_images()
        self._get_anchors()
        self._get_text()
        

### 1.3.1. Tests ###

In [6]:
doc = HtmlDocument("http://sprotasov.ru")
doc.get()
doc.parse()
print(doc.text)

assert "just few links" in doc.text, "Error parsing text"
assert "http://sprotasov.ru/images/gb.svg" in doc.images, "Error parsing images"
assert any(p[1] == "https://twitter.com/07C3" for p in doc.anchors), "Error parsing links"

Stanislav Protasov in just few links:  Curriculum vitae  Google Scholar  GitHub  ResearchGate  Публикации в eLibrary  Facebook  LinkedIn  Research with Stas telegram channel  Подкаст "Происхождение видов": telegram , iTunes , RSS  Automatic testing system ( source code )  Книга "Давайте объясню: или зачем программисту математика"  Материалы на ПостНауке  Twitter


## 1.4. [10] Document analysis ##
Complete the code for `HtmlDocumentTextData` class. Implement word and sentence splitting (use any method you can propose e.g. [nlptk](https://www.nltk.org/api/nltk.tokenize.html)). 

**Criteria of success**: 
1. Your `get_word_stats()` method should return `Counter` object.
2. Don't forget to lowercase your words for counting.
3. Sentences should be obtained inside `<body>` tag only.

In [7]:
from collections import Counter
from nltk.corpus import stopwords
from nltk.tokenize import sent_tokenize, RegexpTokenizer
import re

stop_words = set.union(set(stopwords.words('russian')), set(stopwords.words('english')))

class HtmlDocumentTextData:
    
    def __init__(self, url):
        self.doc = HtmlDocument(url)
        self.doc.get()
        self.doc.parse()
    
    def get_sentences(self):
        # print(result)
        tokenizer = RegexpTokenizer(r'\w+')
        result = tokenizer.tokenize(self.doc.text.lower())
        return result
    
    def get_word_stats(self):
        word_count = {}
        words = [w.lower() for w in self.get_sentences() if w.lower() not in stop_words and w.isalpha()]
        for word in words:
            word_count[word] = word_count.get(word, 0) + 1
        return Counter(word_count)

### 1.4.1. Tests ###

In [8]:
doc = HtmlDocumentTextData("https://innopolis.university")
# doc.get_sentences()
print(doc.get_word_stats().most_common(10))
assert [x for x in doc.get_word_stats().most_common(10) if x[0] == 'иннополис'], 'иннополис should be among most common'

[('иннополис', 20), ('университет', 12), ('области', 10), ('ит', 10), ('лаборатория', 10), ('университета', 9), ('центр', 9), ('технологий', 7), ('робототехники', 7), ('образование', 7)]


## 1.5. [15] Crawling ##

Method `crawl_generator()` is given starting url (`source`) and max depth of search. It should return a **generator** of `HtmlDocumentTextData` objects (return a document as soon as it is downloaded and parsed). You can benefit from `yield obj_name` python construction. Use `HtmlDocumentTextData.anchors` field to go deeper.

In [9]:
from queue import Queue

class Crawler:
    
    def crawl_generator(self, source, depth=1):
        visited = {}
        q = Queue()
        q.put((source, depth))
        visited[source] = True
        while(not q.empty()):
            url, dep = q.get()
            try:
                site = HtmlDocumentTextData(url)
                yield site
            except:
                continue
                
            if(dep == 0):
                continue
            for _, new_url in site.doc.anchors:
                if not visited.get(new_url):
                    q.put((new_url, dep - 1))
                    visited[new_url] = True
        

### 1.5. Tests ###

In [10]:
crawler = Crawler()
counter = Counter()

for c in crawler.crawl_generator("https://innopolis.university/en/", 2):
    print(c.doc.url)
    if c.doc.url[-4:] in ('.pdf', '.mp3', '.avi', '.mp4', '.txt'):
        print("Skipping", c.doc.url)
        continue
    counter.update(c.get_word_stats())
    print(len(counter), "distinct word(s) so far")
    
print("Done")

print(counter.most_common(20))
assert [x for x in counter.most_common(20) if x[0] == 'innopolis'], 'innopolis sould be among most common'

https://innopolis.university/en/
279 distinct word(s) so far
https://apply.innopolis.university/en
906 distinct word(s) so far
https://corporate.innopolis.university/en
1053 distinct word(s) so far
https://media.innopolis.university/en
1097 distinct word(s) so far
https://innopolis.university/lk/
1435 distinct word(s) so far
https://innopolis.university/en/about/
1534 distinct word(s) so far
https://innopolis.university/en/board/
1608 distinct word(s) so far
https://innopolis.university/en/team/
1609 distinct word(s) so far
https://innopolis.university/en/team-structure/
1612 distinct word(s) so far
https://innopolis.university/en/team-structure/education-academics/
1616 distinct word(s) so far
https://innopolis.university/en/team-structure/techcenters/
1618 distinct word(s) so far
https://innopolis.university/en/faculty/
2420 distinct word(s) so far
https://career.innopolis.university/en/job/
2819 distinct word(s) so far
https://career.innopolis.university/en/
3046 distinct word(s) so

Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.
Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.
Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.


https://innopolis.university/public/files/Consent_to_the_processing_of_PD_for_UI.pdf
Skipping https://innopolis.university/public/files/Consent_to_the_processing_of_PD_for_UI.pdf
https://apply.innopolis.university/en/?special=Y
6516 distinct word(s) so far
https://apply.innopolis.university/
6516 distinct word(s) so far
https://apply.innopolis.ru/en/get-in/
6730 distinct word(s) so far
https://apply.innopolis.university/en/faq/
6733 distinct word(s) so far
https://apply.innopolis.university/en/#block5327
6733 distinct word(s) so far
https://apply.innopolis.university/en/olympiad-bonus/
6754 distinct word(s) so far
https://www.ets.org/s/cv/toefl/at-home/
6754 distinct word(s) so far
https://englishtest.duolingo.com/
6754 distinct word(s) so far
http://nic.gov.ru/en/proc/nic/legalize
6828 distinct word(s) so far
https://www.hcch.net/en/instruments/conventions/authorities1/?cid=41
6896 distinct word(s) so far
https://drive.google.com/file/d/10spw7SYKomHSyWuaNMWMl3-t_OiiR-qg/view
6896 dist

Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.
Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.
Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.


https://apply.innopolis.ru/get-in/tests/
7397 distinct word(s) so far
https://innopolis.university/en/campus/
7397 distinct word(s) so far
https://innopolis.com/en/
7499 distinct word(s) so far
https://apply.innopolis.university/public/files/Consent_to_the_processing_of_PD_for_UI.pdf
Skipping https://apply.innopolis.university/public/files/Consent_to_the_processing_of_PD_for_UI.pdf
http://www.campuslife.innopolis.ru/main
7499 distinct word(s) so far
https://corporate.innopolis.university/en/outsourcingrd/
7510 distinct word(s) so far
https://corporate.innopolis.university/en/technologicalaudit/
7558 distinct word(s) so far
https://corporate.innopolis.university/en/stratsession/
7602 distinct word(s) so far
https://corporate.innopolis.university/en/Bootcamp/
7645 distinct word(s) so far
https://corporate.innopolis.university/en/education/
7651 distinct word(s) so far
https://corporate.innopolis.university/
7814 distinct word(s) so far


...

https://university.innopolis.ru/upload/iblock/0b2/AR_2019.pdf
Skipping https://university.innopolis.ru/upload/iblock/0b2/AR_2019.pdf


Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.
Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.
Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.


https://university.innopolis.ru/upload/iblock/189/AR2020(3).pdf
Skipping https://university.innopolis.ru/upload/iblock/189/AR2020(3).pdf
https://university.innopolis.ru/public/files/Consent_to_the_processing_of_PD_for_UI.pdf
Skipping https://university.innopolis.ru/public/files/Consent_to_the_processing_of_PD_for_UI.pdf
http://www.campuslife.innopolis.ru/handbook2021
93216 distinct word(s) so far
http://www.campuslife.innopolis.ru/services
93218 distinct word(s) so far
https://bit.ly/2IX3Nx9
93218 distinct word(s) so far
http://campuslife.innopolis.ru/innostudents
93230 distinct word(s) so far
http://campuslife.innopolis.ru/battle_rage
93254 distinct word(s) so far
https://t.me/joinchat/DjhyZkBN-FmZStxTB40qwQ
93256 distinct word(s) so far
https://docs.google.com/document/d/1D23NYEsWNRexYgeYT8Om3oaNVyGealNYfqOaYlqV4xs/edit
93263 distinct word(s) so far
http://www.campuslife.innopolis.ru/volunteering
93265 distinct word(s) so far
http://www.campuslife.innopolis.ru/aboutsu
93273 distinct 

Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.
Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.
Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.


http://globalai.innopolis.university/
93681 distinct word(s) so far
https://media.innopolis.university/news/grant-na-obuchenie-v-universitete-innopolis-detali-konkursnogo-otbora/
93709 distinct word(s) so far
https://media.innopolis.university/news/blockchain-platform-iu/
93731 distinct word(s) so far
https://media.innopolis.university/public/files/Согласие на обработку ПДн для УИ.pdf
Skipping https://media.innopolis.university/public/files/Согласие на обработку ПДн для УИ.pdf
https://us02web.zoom.us/j/9660990045?pwd=haAZQtiWaT-BVj-FamfQPy-KdnNBvQ
93731 distinct word(s) so far
https://us02web.zoom.us/j/86924417470?pwd=cWpHUlk4OG5BcVNjek94Qk9mMEFjQT09
93731 distinct word(s) so far
https://us02web.zoom.us/j/82115193290?pwd=aStDSVpXUnV5Q0JwZXB3Lys4YmJQUT09
93731 distinct word(s) so far
https://us02web.zoom.us/j/81023939550?pwd=ZHZka0hGTVh4RWFoOUZ3V3lESjVYUT09
93731 distinct word(s) so far
https://us02web.zoom.us/j/84204806112?pwd=Q0ROZkw3bmVGUTB2Q3pYRjFPRWk0UT09
93731 distinct word(s) so 